In [ ]:
import functools
from frozendict import frozendict
import msgpack
import sqlalchemy
sql = sqlalchemy.sql

In [ ]:
class Column(frozendict):

    def __init__(self, name):
        super().__init__(dict(expr='column', name=name))

    def __repr__(self):
        return self['name']

class Equals(frozendict):

    def __init__(self, column, value):
        super().__init__(dict(expr='equals', column=column, value=value))

    def __repr__(self):
        return '{} == {}'.format(self['column'], self['value'])

class And(frozendict):

    def __init__(self, expressions):
        super().__init__(dict(expr='and', data=frozenset(expressions)))

    def __repr__(self):
        return '({})'.format(' & '.join(repr(e) for e in self['data']))
        # return '<And {}>'.format(repr(set(self['data'])))

class Or(frozendict):

    def __init__(self, expressions):
        super().__init__(dict(expr='or', data=frozenset(expressions)))

    def __repr__(self):
        return '({})'.format(' || '.join(repr(e) for e in self['data']))
        # return '<Or {}>'.format(repr(set(self['data'])))

class Not(frozendict):

    def __init__(self, expression):
        super().__init__(dict(expr='not', data=expression))

    def __repr__(self):
        return '~{}'.format(repr(self['data']))
        # return '<Not {}>'.format(repr(self['data']))

def encode(obj):
    ''' Handle frozen things. Note that since order of iteration over a set is arbitrary,
    byte representation will not be consistent. '''
    if isinstance(obj, frozendict):
        return dict(obj)
    if isinstance(obj, frozenset):
        return list(obj)
    return obj

def decode(obj):
    if obj['expr'] == 'column':
        return Column(obj['name'])
    if obj['expr'] == 'equals':
        return Equals(obj['column'], obj['value'])
    if obj['expr'] == 'and':
        return And(obj['data'])
    if obj['expr'] == 'or':
        return Or(obj['data'])
    if obj['expr'] == 'not':
        return Not(obj['data'])
    return obj

packb = functools.partial(msgpack.packb, default=encode, use_bin_type=False)
unpackb = functools.partial(msgpack.unpackb, object_hook=decode, encoding='utf-8')

In [ ]:
obj = Or([
    And([Equals(Column('a'), 1)]),
    Not(And([Equals(Column('b'), 2), Equals(Column('c'), 1)]))])
print(obj)
print(unpackb(packb(obj)))
print(obj == unpackb(packb(obj)))
print(packb(obj))

In [ ]:
class SQLAlchemyConverter(object):

    def __init__(self, column_map):
        self._column_map = column_map

    def convert(self, obj):
        if isinstance(obj, Column):
            return self._column_map[obj]
        if isinstance(obj, Equals):
            return self.convert(obj['column']) == self.convert(obj['value'])
        if isinstance(obj, And):
            return sql.and_(*(self.convert(clause) for clause in obj['data']))
        if isinstance(obj, Or):
            return sql.or_(*(self.convert(clause) for clause in obj['data']))
        if isinstance(obj, Not):
            return sql.not_(self.convert(obj['data']))
        return obj

In [ ]:
metadata = sqlalchemy.MetaData()
table = sqlalchemy.Table('mytable', metadata,
    sqlalchemy.Column('a', sqlalchemy.Integer),
    sqlalchemy.Column('b', sqlalchemy.Integer),
    sqlalchemy.Column('c', sqlalchemy.Integer))
converter = SQLAlchemyConverter({Column(colname): sqlcol for colname, sqlcol in table.columns.items()})

print(str(sql.select(columns=table.columns, whereclause=converter.convert(obj))))